In [2]:
import glob
import pickle
import numpy as np
from music21 import converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from io import BytesIO
from IPython.display import display, HTML
import pandas as pd
from sklearn import preprocessing
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.datasets import imdb
from keras.callbacks import EarlyStopping
from keras.layers import Flatten

/Users/xiechuxi/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Train a LSTM to generate music 
music 21 is a module used for dealing with music data
Step1: we use a special music sheet data format which includes instruments, notes and chords called midi
We load the data from a file with a famous japanese composer joe hisaishi's music 
Using the converter.parse function of music 21, we will get a list of all notes and chords in the file.
We append the pitch of the every note object using its string notation
And we append every chord by encoding the id of chords' notes together into one string with each note seperated by a slash.
notes is the processed data. 

In [48]:
notes=[]
offsets=[]
duration=[]
composite=[]
for file in glob.glob("/Users/xiechuxi/Desktop/music/japan/*.mid"):
    midi = converter.parse(file)
    notes_to_parse = None
    parts = instrument.partitionByInstrument(midi)
    if parts: # file has instrument parts
        notes_to_parse = parts.parts[0].recurse()
    else: # file has notes in flat 
        notes_to_parse = midi.flat.notes
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            composite.append([str(element.pitch),element.quarterLength,element.offset])
        elif isinstance(element, chord.Chord):
            composite.append(['|'.join(str(n) for n in element.normalOrder),element.quarterLength,element.offset])
            
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            duration.append(element.quarterLength)
        elif isinstance(element, chord.Chord):
            duration.append(element.quarterLength)
            
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            offsets.append(element.offset)
        elif isinstance(element, chord.Chord):
            offsets.append(element.offset)
            
    
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('|'.join(str(n) for n in element.normalOrder))



['2|5|9',
 '10|0|2|3',
 'F5',
 'G5',
 'C6',
 '9|2',
 '2|3|7|10',
 'F5',
 'D5',
 'C5',
 'B-4',
 '9|2',
 '10|0|2|3',
 'F5',
 'D5',
 '9|2',
 'C5',
 'A4',
 '3|7|10',
 'A4',
 'G4',
 'A4',
 'G4',
 'A4',
 'G4',
 'G4',
 'G4',
 'A4',
 'G4',
 'A4',
 'A4',
 'G4',
 'A4',
 'G4',
 'A4',
 'A4',
 'G4',
 'A4',
 'G4',
 'A4',
 'G4',
 'A4',
 'G4',
 'G4',
 'A4',
 'F4',
 'A4',
 'G4',
 'D4',
 'A4',
 'G4',
 'G4',
 'G4',
 'B-4',
 'G4',
 'B-4',
 'G4',
 'A4',
 'G4',
 'A4',
 'G4',
 'A4',
 'G4',
 'A4',
 'G4',
 'G4',
 'F4',
 'E-4',
 'B-4',
 'C5',
 'D5',
 'D4',
 'D5',
 'D5',
 'B-4',
 'C5',
 'D5',
 'D5',
 'D5',
 'F5',
 'E-5',
 'D5',
 'C5',
 'B-4',
 'D5',
 'D5',
 'D5',
 'B-4',
 'C5',
 '2',
 'D5',
 'D5',
 '2|5',
 'E-5',
 'D5',
 'C5',
 'B-4',
 'D5',
 'D4',
 'D5',
 'D5',
 'F4',
 '9|0|3',
 'F4',
 'A4',
 'F4',
 'A4',
 'F4',
 'D4',
 'F4',
 'D4',
 'D5',
 'D5',
 'D5',
 'D5',
 'C5',
 'C5',
 'D5',
 'D4',
 'D5',
 'D5',
 '0|2',
 'C5',
 'C5',
 'A4',
 'G4',
 'A4',
 'G4',
 'A4',
 'G4',
 'G4',
 'G4',
 'A4',
 'G4',
 'A4',
 'G4',
 'B-4

Now we get a stream of notes and chords indicated by their notes as characters and chords as number string seperated by '|' (i.e. 1|6|8).
For the following step we will transfer the notes stream of charaters and chords into indices. 

In [49]:
# get all pitch names
pitchnames = sorted(set(item for item in notes))

#map the pitch names into integers
noteindices = dict((note, number) for number, note in enumerate(pitchnames))
durationvarious=sorted(set(item for item in duration))
durationdict = dict((durations,number)for number, durations in enumerate(durationvarious))
#reprocess the whole list into integer indices and seperate the list into input list and outout list we want to predict the next note after each 15 notes 
#but the sequence length could be changed to find the best tune for predict a melody
inputnetcomposite = []
outputnetnote = []
outputnetduration =[]
outputnetoffset=[]
inputnetcomposite1=[]
h=[]
sequence_length = 15


for i in range(0, len(notes) - sequence_length, 1):
    sequencenotein = notes[i:i + sequence_length]
    sequencenoteout = notes[i + sequence_length]
    outputnetnote.append(noteindices[notes[i + sequence_length]])
    outputnetduration.append(durationdict[duration[i + sequence_length]])
    outputnetoffset.append(offsets[i + sequence_length])
    for char in sequencenotein:
        h.append(noteindices[char])
    for j in range(i,i+sequence_length):
        inputnetcomposite.append([h[j],durationdict[duration[j]]])
    for k in range(i,i+sequence_length):
        inputnetcomposite1.append([duration[k],offsets[k]])
        
        
#reshape the input and output net into a format compatible with the LSTM net
inputnetcomposite = np.reshape(inputnetcomposite, (int(len(inputnetcomposite)/sequence_length), sequence_length, 2))

#normalize the numerical value into categorical value
outputnetnote = np_utils.to_categorical(outputnetnote)

outputnetduration = np_utils.to_categorical(outputnetduration)




The function is notes,offset,duration to notes

In [50]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb
from keras.callbacks import EarlyStopping
import numpy as np
from keras.layers import Flatten

# split the input and output nets into test and train set
testinputnetcomposite = inputnetcomposite[round(4/5*len(inputnetcomposite)):len(inputnetcomposite)]
testoutputnetnote = outputnetnote[round(4/5*len(inputnetcomposite)):len(inputnetcomposite)]
traininputnetcomposite = inputnetcomposite[0:round(4/5*len(inputnetcomposite))]
trainoutputnetnote = outputnetnote[0:round(4/5*len(inputnetcomposite))]

print('Build model...')
model1 = Sequential()
model1.add(LSTM(300, dropout=0.0, recurrent_dropout=0.0,input_shape=(traininputnetcomposite.shape[1], traininputnetcomposite.shape[2])))
model1.add(Dense(150, activation='relu'))
model1.add(Dense(len(set(notes))))
model1.add(Activation('softmax'))
model1.compile(loss='categorical_crossentropy', optimizer='adam')
monitor = EarlyStopping(monitor='loss', min_delta=1e-3, patience=10, verbose=1, mode='auto')
print('Train...')

model1.fit(traininputnetcomposite,trainoutputnetnote,validation_data=(testinputnetcomposite,testoutputnetnote),callbacks=[monitor],verbose=2,epochs=400)

Build model...
Train...
Train on 9406 samples, validate on 2351 samples
Epoch 1/400
 - 11s - loss: 4.6143 - val_loss: 4.5880
Epoch 2/400
 - 11s - loss: 4.5006 - val_loss: 4.6230
Epoch 3/400
 - 11s - loss: 4.4560 - val_loss: 4.6094
Epoch 4/400
 - 12s - loss: 4.4130 - val_loss: 4.6134
Epoch 5/400
 - 12s - loss: 4.3655 - val_loss: 4.6679
Epoch 6/400
 - 10s - loss: 4.3180 - val_loss: 4.7198
Epoch 7/400
 - 10s - loss: 4.2732 - val_loss: 4.7168
Epoch 8/400
 - 10s - loss: 4.2279 - val_loss: 4.7878
Epoch 9/400
 - 10s - loss: 4.1936 - val_loss: 4.8495
Epoch 10/400
 - 10s - loss: 4.1571 - val_loss: 4.8668
Epoch 11/400
 - 10s - loss: 4.1292 - val_loss: 4.9600
Epoch 12/400
 - 10s - loss: 4.1001 - val_loss: 5.1507
Epoch 13/400
 - 10s - loss: 4.0743 - val_loss: 5.0946
Epoch 14/400
 - 10s - loss: 4.0544 - val_loss: 5.0943
Epoch 15/400
 - 10s - loss: 4.0281 - val_loss: 5.2173
Epoch 16/400
 - 10s - loss: 3.9985 - val_loss: 5.2007
Epoch 17/400
 - 10s - loss: 3.9683 - val_loss: 5.0895
Epoch 18/400
 - 10s

Epoch 151/400
 - 10s - loss: 1.5000 - val_loss: 9.3905
Epoch 152/400
 - 10s - loss: 1.5027 - val_loss: 9.2915
Epoch 153/400
 - 10s - loss: 1.4849 - val_loss: 9.5173
Epoch 154/400
 - 10s - loss: 1.4770 - val_loss: 9.4254
Epoch 155/400
 - 10s - loss: 1.4685 - val_loss: 9.4413
Epoch 156/400
 - 10s - loss: 1.4716 - val_loss: 9.4393
Epoch 157/400
 - 10s - loss: 1.4634 - val_loss: 9.5224
Epoch 158/400
 - 10s - loss: 1.4512 - val_loss: 9.4306
Epoch 159/400
 - 10s - loss: 1.4539 - val_loss: 9.4185
Epoch 160/400
 - 10s - loss: 1.4380 - val_loss: 9.5036
Epoch 161/400
 - 10s - loss: 1.4067 - val_loss: 9.5950
Epoch 162/400
 - 10s - loss: 1.4084 - val_loss: 9.7457
Epoch 163/400
 - 10s - loss: 1.3990 - val_loss: 9.7047
Epoch 164/400
 - 10s - loss: 1.3991 - val_loss: 9.6392
Epoch 165/400
 - 10s - loss: 1.4158 - val_loss: 9.6671
Epoch 166/400
 - 10s - loss: 1.3991 - val_loss: 9.4060
Epoch 167/400
 - 10s - loss: 1.4060 - val_loss: 9.6364
Epoch 168/400
 - 10s - loss: 1.3657 - val_loss: 9.7274
Epoch 169/

 - 10s - loss: 0.8040 - val_loss: 11.3730
Epoch 299/400
 - 10s - loss: 0.8335 - val_loss: 11.1559
Epoch 300/400
 - 10s - loss: 0.8061 - val_loss: 11.3607
Epoch 301/400
 - 10s - loss: 0.7768 - val_loss: 11.1792
Epoch 302/400
 - 10s - loss: 0.7972 - val_loss: 11.2233
Epoch 303/400
 - 10s - loss: 0.9027 - val_loss: 11.1657
Epoch 304/400
 - 10s - loss: 0.7965 - val_loss: 11.3284
Epoch 305/400
 - 10s - loss: 0.8325 - val_loss: 11.4330
Epoch 306/400
 - 10s - loss: 0.8349 - val_loss: 11.2774
Epoch 307/400
 - 10s - loss: 0.7967 - val_loss: 11.4358
Epoch 308/400
 - 10s - loss: 0.8025 - val_loss: 11.3858
Epoch 309/400
 - 10s - loss: 0.7840 - val_loss: 11.6404
Epoch 310/400
 - 10s - loss: 0.7701 - val_loss: 11.4610
Epoch 311/400
 - 10s - loss: 0.7645 - val_loss: 11.4113
Epoch 312/400
 - 10s - loss: 0.7960 - val_loss: 11.2658
Epoch 313/400
 - 10s - loss: 0.8192 - val_loss: 11.3969
Epoch 314/400
 - 10s - loss: 0.8963 - val_loss: 11.2799
Epoch 315/400
 - 10s - loss: 0.7960 - val_loss: 11.2899
Epoch 

The function is offset,notes,duration to duration

In [52]:
# split the input and output nets into test and train set

testinputnetcomposite = inputnetcomposite[round(4/5*len(inputnetcomposite)):len(inputnetcomposite)]
testoutputnetduration = outputnetduration[round(4/5*len(inputnetcomposite)):len(inputnetcomposite)]
traininputnetcomposite = inputnetcomposite[0:round(4/5*len(inputnetcomposite))]
trainoutputnetduration = outputnetduration[0:round(4/5*len(inputnetcomposite))]
print('Build model...')
model2 = Sequential()
model2.add(LSTM(300, dropout=0.0, recurrent_dropout=0.0,input_shape=(traininputnetcomposite.shape[1], traininputnetcomposite.shape[2])))
model2.add(Dense(150, activation='relu'))
model2.add(Dense(54))
model2.add(Activation('softmax'))
model2.compile(loss='categorical_crossentropy', optimizer='adam')
monitor = EarlyStopping(monitor='loss', min_delta=1e-3, patience=10, verbose=1, mode='auto')
print('Train...')

model2.fit(traininputnetcomposite,trainoutputnetduration,validation_data=(testinputnetcomposite,testoutputnetduration),callbacks=[monitor],verbose=2,epochs=400)

Build model...
Train...
Train on 9406 samples, validate on 2351 samples
Epoch 1/400
 - 12s - loss: 1.8232 - val_loss: 2.5082
Epoch 2/400
 - 11s - loss: 1.5807 - val_loss: 2.6226
Epoch 3/400
 - 10s - loss: 1.4969 - val_loss: 2.5086
Epoch 4/400
 - 9s - loss: 1.4329 - val_loss: 2.5735
Epoch 5/400
 - 10s - loss: 1.3889 - val_loss: 2.6404
Epoch 6/400
 - 9s - loss: 1.3466 - val_loss: 2.5873
Epoch 7/400
 - 9s - loss: 1.3205 - val_loss: 2.4544
Epoch 8/400
 - 9s - loss: 1.2881 - val_loss: 2.3168
Epoch 9/400
 - 9s - loss: 1.2650 - val_loss: 2.4550
Epoch 10/400
 - 10s - loss: 1.2414 - val_loss: 2.1845
Epoch 11/400
 - 10s - loss: 1.2239 - val_loss: 2.3123
Epoch 12/400
 - 10s - loss: 1.2091 - val_loss: 2.3183
Epoch 13/400
 - 10s - loss: 1.2034 - val_loss: 2.1285
Epoch 14/400
 - 10s - loss: 1.1797 - val_loss: 2.0961
Epoch 15/400
 - 10s - loss: 1.1703 - val_loss: 2.2795
Epoch 16/400
 - 9s - loss: 1.1545 - val_loss: 2.1871
Epoch 17/400
 - 9s - loss: 1.1449 - val_loss: 2.3315
Epoch 18/400
 - 9s - loss:

Epoch 153/400
 - 9s - loss: 0.3576 - val_loss: 3.8293
Epoch 154/400
 - 9s - loss: 0.3419 - val_loss: 4.1557
Epoch 155/400
 - 9s - loss: 0.3586 - val_loss: 3.7935
Epoch 156/400
 - 9s - loss: 0.3726 - val_loss: 4.2622
Epoch 157/400
 - 9s - loss: 0.4165 - val_loss: 3.9887
Epoch 158/400
 - 9s - loss: 0.3904 - val_loss: 4.0109
Epoch 159/400
 - 9s - loss: 0.3406 - val_loss: 3.9883
Epoch 160/400
 - 9s - loss: 0.3343 - val_loss: 4.0328
Epoch 161/400
 - 9s - loss: 0.3240 - val_loss: 4.2620
Epoch 162/400
 - 10s - loss: 0.3172 - val_loss: 4.2995
Epoch 163/400
 - 9s - loss: 0.3414 - val_loss: 3.9948
Epoch 164/400
 - 10s - loss: 0.3475 - val_loss: 4.4030
Epoch 165/400
 - 10s - loss: 0.3477 - val_loss: 4.5100
Epoch 166/400
 - 9s - loss: 0.3207 - val_loss: 4.2869
Epoch 167/400
 - 9s - loss: 0.3119 - val_loss: 4.2835
Epoch 168/400
 - 10s - loss: 0.3039 - val_loss: 4.3547
Epoch 169/400
 - 9s - loss: 0.3724 - val_loss: 3.9803
Epoch 170/400
 - 9s - loss: 0.3628 - val_loss: 3.9418
Epoch 171/400
 - 9s - lo

The function is duration,offset to offset

In [53]:
# split the input and output nets into test and train set
testinputnetcomposite1 = inputnetcomposite1[round(4/5*len(inputnetcomposite1)):len(inputnetcomposite1)]
testoutputnetoffset = outputnetoffset[round(4/5*len(inputnetcomposite1)):len(inputnetcomposite1)]
traininputnetcomposite1 = inputnetcomposite1[0:round(4/5*len(inputnetcomposite1))]
trainoutputnetoffset = outputnetoffset[0:round(4/5*len(inputnetcomposite1))]

print('Build model...')
model3 = Sequential()
model3.add(LSTM(300, dropout=0.0, recurrent_dropout=0.0,input_shape=(traininputnetcomposite1.shape[1], traininputnetcomposite1.shape[2])))
model3.add(Dense(150, activation='relu'))
model3.add(Dense(1,activation='relu'))
model3.compile(loss='mean_squared_error', optimizer='adam')
monitor = EarlyStopping(monitor='loss', min_delta=1e-3, patience=10, verbose=1, mode='auto')
print('Train...')

model3.fit(traininputnetcomposite1,trainoutputnetoffset,validation_data=(testinputnetcomposite1,testoutputnetoffset),callbacks=[monitor],verbose=2,epochs=400)


Build model...


AttributeError: 'list' object has no attribute 'shape'

In [74]:
from numpy import vstack
from fractions import Fraction
seed = np.random.randint(190, size=(15, 2))
#first generate a list of random notes if you want to be more creative with melody 
#you could write a small piece of notes by yourself or copy a piece online.
sf=np.random.randint(len(inputnetcomposite)-15)
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
int_to_duration = dict((number, durations) for number, durations in enumerate(durationvarious))
pattern = inputnetcomposite1[sf:sf+15] #pattern is the iterator which each iteration will drop the first note and predict the next note according to the pattern
predictoutput=[]
#generate 500 notes as seed so that following melody could be created based on this seed
for i in range(400):
    predictinput = np.reshape(pattern, (1, len(pattern), 2))
    predictionnote = model1.predict(predictinput, verbose=0)
    predictionduration = model2.predict(predictinput,verbose=0)
    predictionduration1=np.argmin(predictionduration)
    index = np.argmax(predictionnote)
    result = int_to_note[index]
    resultduration=int_to_duration[predictionduration1]
    index = np.array([[index,predictionduration1]])
    predictoutput.append([result,resultduration])
    pattern= vstack((pattern, index))
    pattern = pattern[1:len(pattern)]



In [77]:
offset = 0
output_notes = []

for l in range(0,len(predictoutput)):
    # pattern is a chord
    if ('|' in predictoutput[l][0]) or predictoutput[l][0].isdigit():
        notes_in_chord = predictoutput[l][0].split('|')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            new_note.quarterLength=predictoutput[l][1]
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(predictoutput[l][0])
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        new_note.quarterLength=predictoutput[l][1]
        #we can actually change the instrument in the further continuation of the project
        output_notes.append(new_note)
    
    offset+=0.3
    #to make sure notes don't stack

In [78]:
from music21 import *
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='/Users/xiechuxi/Desktop/music/musiccreate/gn16.mid')

'/Users/xiechuxi/Desktop/music/musiccreate/gn16.mid'

AttributeError: 'list' object has no attribute 'show'